# Open Street Map Jogging route planner

In [1]:
import requests
import json
from dataclasses import dataclass
from typing import Dict, Tuple
import xml.etree.ElementTree as ET
import math
import random
import heapq

In [2]:
# 1. get Street and Node data from OSM

In [3]:
#define dataclasses to store the data organized
@dataclass
class Node:
    id: int
    lat: float
    lon: float
    edges: list
    
    def __str__(self) -> str:
        return f"Node {self.id} ({self.lat}, {self.lon}), {len(self.edges)} edges"

@dataclass
class Edge:
    id: int
    start: Node
    end: Node
    length: float
    
    def __init__(self, id, start, end, length):
        self.id = id
        self.start = start
        self.end = end
        self.length = length
        
        start.edges.append(self)
        end.edges.append(self)
    
    def __str__(self) -> str:
        return f"Edge {self.id} ({self.start} -> {self.end})"

In [4]:
# get data from OSM
# store the data in the dataclasses
def get_data(pos1: Tuple[float, float], pos2: Tuple[float, float] ) -> Tuple[Dict[int, Node], Dict[int, Edge]]:
    """Get data from OSM and store it in the dataclasses"""
    
    min_lat = min(pos1[0], pos2[0])
    max_lat = max(pos1[0], pos2[0])
    min_lon = min(pos1[1], pos2[1])
    max_lon = max(pos1[1], pos2[1])
    
    URL = f"https://api.openstreetmap.org/api/0.6/map?bbox={min_lon},{min_lat},{max_lon},{max_lat}"
    raw_data = requests.get(URL).text
    
    nodes = {}  # id -> Node
    edges = {}  # id -> Edge
    
    # Parse nodes
    tree = ET.fromstring(raw_data)
    for node_elem in tree.iter("node"):
        node_id = int(node_elem.get("id"))
        node_lat = float(node_elem.get("lat"))
        node_lon = float(node_elem.get("lon"))
        node_edges = []
        nodes[node_id] = Node(id=node_id, lat=node_lat, lon=node_lon, edges=node_edges)
    
    # Parse edges
    for way_elem in tree.iter("way"):
        way_id = int(way_elem.get("id"))
        way_nodes = way_elem.findall("nd")
        way_length = 0.0
        start_node = None
        end_node = None
        
        for i in range(len(way_nodes) - 1):
            start_node_id = int(way_nodes[i].get("ref"))
            end_node_id = int(way_nodes[i+1].get("ref"))
            
            if start_node_id not in nodes or end_node_id not in nodes:
                continue
            
            start_node = nodes[start_node_id]
            end_node = nodes[end_node_id]
            
            distance = math.sqrt((start_node.lat - end_node.lat)**2 + (start_node.lon - end_node.lon)**2)
            way_length += distance
            
            edge = Edge(id=way_id, start=start_node, end=end_node, length=distance)
            edges[way_id] = edge
            
        if start_node and end_node:
            start_node.edges.append(edge)
            end_node.edges.append(edge)
    
    return nodes, edges

In [5]:
def find_closest_node(nodes: Dict[int, Node], pos: Tuple[float, float]) -> Node:
    min_dist = float("inf")
    for node in nodes.values():
        dist = haversine(pos[0], pos[1], node.lat, node.lon)
        if dist < min_dist:
            min_dist = dist
            closest_node = node
    return closest_node

def haversine(lat_1, lon_1, lat_2, lon_2) -> float:
    #returns the distance between two points in km
    # can later be replaced by a more accurate function that takes the curvature of the earth into account
    return math.sqrt((lat_1 - lat_2)**2 + (lon_1 - lon_2)**2)

def Distance_Constrained_Route(nodes: Dict[int, Node], edges: Dict[int, Edge], start_pos: Tuple[float, float], length: float) -> Tuple[float, list]:
    """Calculate a route with a given length that starts and returns to the same point. No Edge can be taken twice."""
    visited = set()
    path = []
    total_length = 0
    
    start_node = find_closest_node(nodes, start_pos)
    
    current_node = start_node
    path.append(current_node.id)
    visited.add(current_node.id)
    
    random_way_length = 2 * length / 3 # two thirds of the length
    while total_length < random_way_length:
        unvisited_nodes = [node for node in current_node.edges if node.id not in visited]
        
        if not unvisited_nodes:
            # Backtracking
            path.pop()
            current_node = nodes[path[-1]] if path else start_node
            continue
        
        next_node = random.choice(unvisited_nodes)
        distance = math.sqrt((current_node.lat - next_node.lat)**2 + (current_node.lon - next_node.lon)**2)
        
        total_length += distance
        current_node = next_node
        
        visited.add(current_node.id)
        path.append(current_node.id)
        
    # Find the way back with the remaining length
    while total_length < length:
        unvisited_nodes = [node for node in current_node.edges if node.id not in visited]
        
        if not unvisited_nodes:
            # Backtracking
            path.pop()
            current_node = nodes[path[-1]] if path else start_node
            continue
        
        next_node = min(unvisited_nodes, key=lambda node: haversine(node.lat, node.lon, start_node.lat, start_node.lon))
        distance = math.sqrt((current_node.lat - next_node.lat)**2 + (current_node.lon - next_node.lon)**2)
        
        total_length += distance
        current_node = next_node
        
        visited.add(current_node.id)
        path.append(current_node.id)
    
    # Calculate the deviation
    deviation = abs(total_length - length)
    return deviation, path

In [6]:
nodes = {} #Dict[id: int, Node]
edges = {} #Dict[id: int, Edge]

#set the bounding box to Hamburg, Germany

pos_1 = (53.695, 9.755)
pos_2 = (53.695001, 10.755001)
nodes, edges = get_data(pos_1, pos_2)

print(len(nodes), len(edges))

start = random.choice(list(nodes.values()))

278 16


In [7]:
deviation, route = Distance_Constrained_Route(nodes, edges, (start.lat, start.lon), 5.0)

AttributeError: 'Edge' object has no attribute 'lat'